In [1]:
import pandas as pd
import numpy as np
from plotly.graph_objs import *
from pyecharts import options as opts
from pyecharts.charts import Kline, Line

In [2]:
df = pd.read_csv("ETH_daily.csv")  

In [3]:
df["Close_1_shift"] = df["Close"].shift(1)  

In [4]:
df["TR"] = np.abs(df.High - df.Low)
df["TR"] = np.maximum(
    df["TR"],
    np.maximum(
        np.abs(df.Close_1_shift - df.High),
        np.abs(df.Close_1_shift - df.Low),
    ),
)

In [5]:
n_array = np.array(df["TR"].values)
n_array[20] = np.mean(df["TR"][:20])
for i in range(21, df.shape[0]):
    n_array[i] = (19.0 * n_array[i - 1] + df["TR"][i]) / 20.0
df["N"] = n_array


In [6]:
# Compute upper and lower bounds based on Turtle Algorithm
df["upper_bound"] = df["High"].shift(1).rolling(window=20).max()
df["lower_bound"] = df["Low"].shift(1).rolling(window=10).min()


In [7]:
crypto_value = [0 for i in range(20)]
money = [1000 for i in range(20)]
init_money = 1000
crypto = 0
fees = 0.001  # fees as 0.1%
positions = []  
success_history = []  
failure_history = []  

#original turtle, 考虑仓位管理，每次加仓1 unit

for i in range(21, df.shape[0]): 

    if ( crypto == 0 and df["Close_1_shift"].iloc[i] > df["upper_bound"].iloc[i-1] ): #建仓
        # We will use average price from the current ticker
        price = (df["Close"].iloc[i] + df["Open"].iloc[i]) / 2.0
        unit = 1 *0.01*init_money/df["N"].iloc[i]

        if money[-1] > unit*price*(1+fees):    #剩下的钱够买 1 unit
            crypto += unit
            crypto_value+= [crypto *df["Close"].iloc[i]]
            money+=[money[-1]-unit * price*(1+fees)]

        elif 0<money[-1] and money[-1]<unit*price:     #剩下的钱不够买 1 unit
            crypto += money[-1]*(1.0 - fees)/price
            crypto_value+= [crypto *df["Close"].iloc[i]]
            money+=[0]
            
        increase_position = price + 0.5*df["N"].iloc[i]
        stop_loss = price - 2.0 * df["N"].iloc[i]
        positions += [{ "date": df.Date.iloc[i], "price": price, "money": init_money}]

    elif crypto == 0:    #空仓不动
        crypto_value += [crypto_value[-1]]
        money += [money[-1]]
        
    elif (crypto > 0 and df["Close_1_shift"].iloc[i] > increase_position and money[-1] >0) : #加仓
        price = (df["Close"].iloc[i] + df["Open"].iloc[i]) / 2.0
        unit = 1 *0.01*init_money/df["N"].iloc[i]
        
        if money[-1] > unit*price*(1+fees):    #剩下的钱够买 1 unit
            crypto += unit
            crypto_value+= [crypto *df["Close"].iloc[i]]
            money+=[money[-1]-unit * price*(1+fees)]

        elif 0<money[-1] and money[-1]<unit*price:     #剩下的钱不够买 1 unit
            crypto += money[-1]*(1.0 - fees)/price
            crypto_value+= [crypto *df["Close"].iloc[i]]
            money+=[0]
            
        increase_position = price + 0.5*df["N"].iloc[i]
        stop_loss = price - 2.0 * df["N"].iloc[i]

        
    elif  crypto > 0 and (
        df["Close_1_shift"].iloc[i]< df["lower_bound"].iloc[i-1] 
        or df["Close_1_shift"].iloc[i] < stop_loss  
        or i == df.shape[0] - 1 ):                      #清仓
        
        price = (df["Close"].iloc[i] + df["Open"].iloc[i]) / 2.0
        money += [money[-1] + crypto * price * (1 - fees)]
        init_money = money[-1]
        stop_loss, crypto = 0, 0
        crypto_value += [0]
        if positions[-1]["price"] < price:
            success_history += [
                {
                    "date": [positions[-1]["date"], df.Date.iloc[i]],
                    "money": [positions[-1]["money"], money[-1]],
                }
            ]
        else:
            failure_history += [
                {
                    "date": [positions[-1]["date"], df.Date.iloc[i]],
                    "money": [positions[-1]["money"], money[-1]],
                }
            ]
        positions = []
        
    elif crypto > 0:           #持仓不动
        crypto_value += [crypto * df["Close"].iloc[i]]
        money += [money[-1]]

rtn1 = [np.round((crypto_value[i]+money[i])/money[0],2) for i in range(df.shape[0]-1)]

In [8]:
money = [1000.0 for i in range(20)]  # initial money value in USDT
crypto_value = [0 for i in range(20)]
crypto = 0  
fees = 0.001  # fees as 0.1%
positions = []  
success_history = []  
failure_history = []  

# 简单版海龟交易，不考虑仓位管理

for i in range(21, df.shape[0]):  

    if ( crypto == 0 and df["Close_1_shift"].iloc[i] > df["upper_bound"].iloc[i-1] ): # 1.突破上轨，建仓

        price = (df["Close"].iloc[i] + df["Open"].iloc[i]) / 2.0
        crypto = np.round(money[-1]*(1.0 - fees)/ price, 4)
        crypto_value += [crypto*df["Close"].iloc[i]]
        money += [0]
        positions += [{ "date": df.Date.iloc[i], "price": price}]
        stop_loss = price - 2.0 * df["N"].iloc[i]  # set stop loss


    elif crypto == 0:                                       # 2.空仓不动
        money += [money[-1]]
        crypto_value += [0]
        
    elif crypto > 0 and (
        df["Close_1_shift"].iloc[i]< df["lower_bound"].iloc[i-1]  # 3、突破下轨或达到止损条件stop_loss或最后一次交易，清仓
        or df["Close_1_shift"].iloc[i] < stop_loss  
        or i == df.shape[0] - 1 ): 

        price = (df["Close"].iloc[i] + df["Open"].iloc[i]) / 2.0
        money += [crypto * price *(1-fees)]
        crypto = 0
        crypto_value +=[0]
        stop_loss = 0.0

        if positions[-1]["price"] < price:
            success_history += [
                {
                    "date": [positions[-1]["date"], df.Date.iloc[i]],
                    "price": [positions[-1]["price"], price],
                }
            ]
        else:
            failure_history += [
                {
                    "date": [positions[-1]["date"], df.Date.iloc[i]],
                    "price": [positions[-1]["price"], price],
                }
            ]
        positions = []
        
    elif crypto > 0:                              # 4.持仓不动
        money += [0]
        crypto_value += [crypto * df["Close"].iloc[i]]

rtn2 = [np.round((crypto_value[i]+money[i])/money[0],2) for i in range(df.shape[0]-1)]

In [9]:
N1 = 10 
N2 = 20
df["ema_fast"] = np.round(df["Close"].ewm(span = N1, adjust=False).mean(),4)
df["ema_slow"] = np.round(df["Close"].ewm(span = N2, adjust=False).mean(),4)

money = [1000.0 for i in range(20)]  # initial money value in USDT
crypto_value = [0 for i in range(20)]
crypto = 0  
fees = 0.001  # fees as 0.1%
positions = []  
success_history = []  
failure_history = []  

for i in range(21, df.shape[0]):
    if crypto == 0 and df["ema_fast"].iloc[i-1] < df["ema_slow"].iloc[i-1] and df["ema_fast"].iloc[i] > df["ema_slow"].iloc[i] and i != df.shape[0] - 1: 

        price = (df["Close"].iloc[i] + df["Open"].iloc[i]) / 2.0
        crypto = np.round(money[-1]*(1.0 - fees)/ price, 4)
        crypto_value += [crypto*df["Close"].iloc[i]]
        money += [0]
        positions += [{ "date": df.Date.iloc[i], "price": price}]
        
    elif crypto == 0:
        money += [money[-1]]
        crypto_value += [0]
            
    elif crypto > 0 and (df["ema_fast"].iloc[i-1] > df["ema_slow"].iloc[i-1] and df["ema_fast"].iloc[i] < df["ema_slow"].iloc[i] or i == df.shape[0] - 1 ):
        
        price = (df["Close"].iloc[i] + df["Open"].iloc[i]) / 2.0
        money += [crypto * price *(1-fees)]
        crypto = 0
        crypto_value +=[0]
        
        if positions[-1]["price"] < price:
            success_history += [
                {
                    "date": [positions[-1]["date"], df.Date.iloc[i]],
                    "price": [positions[-1]["price"], price],
                }
            ]
        else:
            failure_history += [
                {
                    "date": [positions[-1]["date"], df.Date.iloc[i]],
                    "price": [positions[-1]["price"], price],
                }
            ]
        positions = []

    elif crypto > 0:
        money += [0]
        crypto_value += [crypto * df["Close"].iloc[i]]

rtn3 = [np.round((crypto_value[i]+money[i])/money[0],2) for i in range(df.shape[0]-1)]

In [10]:
K = 2                                           
N = 20 
df["sma"] = np.round(df["Close"].rolling(N).mean(),4)
df["stdev"] = np.round(df["Close"].rolling(N).std(),4) 
df["upband"]  = np.round(df["sma"] + K*df["stdev"],4)
df["downband"] = np.round(df["sma"] - K*df["stdev"],4)

money = [1000.0 for i in range(20)]  # initial money value in USDT
crypto_value = [0 for i in range(20)]
crypto = 0 
fees = 0.001           # fees as 0.1%
positions = []  
success_history = []  
failure_history = []  

#布林通道顺势策略

for i in range(21, df.shape[0]):
    if crypto == 0 and df["Close"].iloc[i-1] > df["upband"].iloc[i-1] and i != df.shape[0] - 1:     # 1.突破上轨线，建仓

        price = (df["Close"].iloc[i] + df["Open"].iloc[i]) / 2.0
        crypto = np.round(money[-1]*(1.0 - fees)/ price, 4)
        crypto_value += [crypto*df["Close"].iloc[i]]
        money += [0]
        positions += [{ "date": df.Date.iloc[i], "price": price}]
        
    elif crypto == 0:                                                                    # 2.空仓不动
        money += [money[-1]]
        crypto_value += [0]
            
    elif crypto > 0 and (df["Close"].iloc[i-1] < df["downband"].iloc[i-1] or i == df.shape[0] - 1 ):   # 3.突破下轨线，清仓       
        
        price = (df["Close"].iloc[i] + df["Open"].iloc[i]) / 2.0
        money += [crypto * price *(1-fees)]
        crypto = 0
        crypto_value +=[0]
        
        if positions[-1]["price"] < price:
            success_history += [
                {
                    "date": [positions[-1]["date"], df.Date.iloc[i]],
                    "price": [positions[-1]["price"], price],
                }
            ]
        else:
            failure_history += [
                {
                    "date": [positions[-1]["date"], df.Date.iloc[i]],
                    "price": [positions[-1]["price"], price],
                }
            ]
        positions = []

    elif crypto > 0:                                                             # 4.持仓不动
        money += [0]
        crypto_value += [crypto * df["Close"].iloc[i]]

rtn4 = [np.round((crypto_value[i]+money[i])/money[0],2) for i in range(df.shape[0]-1)]

In [11]:
attr=[str(t) for t in df.Date]

ben = [np.round(j/df.Close.iloc[0],4) for j in df.Close]

line2 = (
    Line()
    .add_xaxis(attr)
    .add_yaxis("rtn_original_turtle",rtn1 , is_smooth = True,)    
    .add_yaxis("rtn_simple_turtle",rtn2 , is_smooth = True,)
    .add_yaxis("rtn_2ema",rtn3 , is_smooth = True,)
    .add_yaxis("rtn_boll",rtn4 , is_smooth = True,) 
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(is_scale=True),
        yaxis_opts=opts.AxisOpts(
            is_scale=True,
            splitarea_opts=opts.SplitAreaOpts(
                is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
            ),
        ),
        datazoom_opts=[opts.DataZoomOpts(pos_bottom="-2%")],
        title_opts=opts.TitleOpts(title="rtn_compare"),
    ) 
)
line2.render("rtn_compare.html")

'F:\\jupyter\\rtn_compare.html'